# **Seq-Nms + Yolov5**

# https://github.com/lrghust/Seq-NMS/blob/master/seqnms.py
# 
# https://github.com/amusi/Non-Maximum-Suppression/blob/master/nms.py

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys

sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
model = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
#                        path='/kaggle/input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',
                       path='../input/1920-yolov5m6-19-cots/best.pt',
                       source='local',
                       force_reload=True)  # local repo
model.conf = 0.00
model.iou = 1
model.max_det = 1000

In [ ]:
CLASSES = ('background', 'starfish')
import time
IOU_THRESH = 0.4
import copy

In [ ]:
def nms(bounding_boxes, confidence_score, threshold):
    # Copied from https://github.com/amusi/Non-Maximum-Suppression/blob/master/nms.py
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes
    score = np.array(confidence_score)

    # Picked bounding boxes
    picked_boxes = []
    picked_score = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])

        # Compute ordinates of intersection-over-union(IOU)
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)

        left = np.where(ratio < threshold)
        order = order[left]

    return picked_boxes, picked_score

In [ ]:
#Copied from https://github.com/lrghust/Seq-NMS/blob/master/seqnms.py
def createLinks(dets_all):
    links_all=[]
    frame_num=len(dets_all[0])
    cls_num=len(CLASSES)-1
    for cls_ind in range(cls_num): 
        links_cls=[] 
#         link_begin=time.time()
        for frame_ind in range(frame_num-1): 
            dets1=dets_all[cls_ind][frame_ind]
            dets2=dets_all[cls_ind][frame_ind+1]
            box1_num=len(dets1)
            box2_num=len(dets2)
            if frame_ind==0:
                areas1=np.empty(box1_num)
                for box1_ind,box1 in enumerate(dets1):
                    areas1[box1_ind]=(box1[2]-box1[0]+1)*(box1[3]-box1[1]+1)
            else: 
                areas1=areas2
            areas2=np.empty(box2_num)
            for box2_ind,box2 in enumerate(dets2):
                areas2[box2_ind]=(box2[2]-box2[0]+1)*(box2[3]-box2[1]+1)
            links_frame=[] 
            for box1_ind,box1 in enumerate(dets1):
                area1=areas1[box1_ind]
                x1=np.maximum(box1[0],dets2[:,0])
                y1=np.maximum(box1[1],dets2[:,1])
                x2=np.minimum(box1[2],dets2[:,2])
                y2=np.minimum(box1[3],dets2[:,3])
                w =np.maximum(0.0, x2 - x1 + 1)
                h =np.maximum(0.0, y2 - y1 + 1)
                inter = w * h
                ovrs = inter / (area1 + areas2 - inter)
                links_box=[ovr_ind for ovr_ind,ovr in enumerate(ovrs) if ovr >= IOU_THRESH]
                links_frame.append(links_box)
            links_cls.append(links_frame)
#         link_end=time.time()
#         print('link: {:.4f}s'.format(link_end - link_begin))
        links_all.append(links_cls)
    return links_all

def maxPath(dets_all,links_all):
    for cls_ind,links_cls in enumerate(links_all):
#         max_begin=time.time()
        dets_cls=dets_all[cls_ind]
        while True:
            rootindex,maxpath,maxsum=findMaxPath(links_cls,dets_cls)
            if len(maxpath) <= 1:
                break
            rescore(dets_cls,rootindex,maxpath,maxsum)
            deleteLink(dets_cls,links_cls,rootindex,maxpath,IOU_THRESH)
#         max_end=time.time()
#         print ('max path: {:.4f}s'.format(max_end - max_begin))

def NMS(dets_all):
    for cls_ind,dets_cls in enumerate(dets_all):
        for frame_ind,dets in enumerate(dets_cls):
            keep=nms(dets, NMS_THRESH)
            dets_all[cls_ind][frame_ind]=dets[keep, :]

def findMaxPath(links,dets):
    maxpaths=[]
    roots=[] 
    maxpaths.append([ (box[4],[ind]) for ind,box in enumerate(dets[-1])])
    for link_ind,link in enumerate(links[::-1]): 
        curmaxpaths=[]
        linkflags=np.zeros(len(maxpaths[0]),int)
        det_ind=len(links)-link_ind-1
        for ind,linkboxes in enumerate(link):
            if linkboxes == []:
                curmaxpaths.append((dets[det_ind][ind][4],[ind]))
                continue
            linkflags[linkboxes]=1
            prev_ind=np.argmax([maxpaths[0][linkbox][0] for linkbox in linkboxes])
            prev_score=maxpaths[0][linkboxes[prev_ind]][0]
            prev_path=copy.copy(maxpaths[0][linkboxes[prev_ind]][1])
            prev_path.insert(0,ind)
            curmaxpaths.append((dets[det_ind][ind][4]+prev_score,prev_path))
        root=[maxpaths[0][ind] for ind,flag in enumerate(linkflags) if flag == 0]
        roots.insert(0,root)
        maxpaths.insert(0,curmaxpaths)
    roots.insert(0,maxpaths[0])
    maxscore=0
    maxpath=[]
    for index,paths in enumerate(roots):
        if paths==[]:
            continue
        maxindex=np.argmax([path[0] for path in paths])
        if paths[maxindex][0]>maxscore:
            maxscore=paths[maxindex][0]
            maxpath=paths[maxindex][1]
            rootindex=index
    return rootindex,maxpath,maxscore

def rescore(dets, rootindex, maxpath, maxsum):
    newscore=maxsum/len(maxpath)
    for i,box_ind in enumerate(maxpath):
        dets[rootindex+i][box_ind][4]=newscore

def deleteLink(dets,links, rootindex, maxpath,thesh):
    for i,box_ind in enumerate(maxpath):
        areas=[(box[2]-box[0]+1)*(box[3]-box[1]+1) for box in dets[rootindex+i]]
        area1=areas[box_ind]
        box1=dets[rootindex+i][box_ind]
        x1=np.maximum(box1[0],dets[rootindex+i][:,0])
        y1=np.maximum(box1[1],dets[rootindex+i][:,1])
        x2=np.minimum(box1[2],dets[rootindex+i][:,2])
        y2=np.minimum(box1[3],dets[rootindex+i][:,3])
        w =np.maximum(0.0, x2 - x1 + 1)
        h =np.maximum(0.0, y2 - y1 + 1)
        inter = w * h
        ovrs = inter / (area1 + areas - inter)
        deletes=[ovr_ind for ovr_ind,ovr in enumerate(ovrs) if ovr >= IOU_THRESH]
        if rootindex+i<len(links):
            for delete_ind in deletes:
                links[rootindex+i][delete_ind]=[]
        if i > 0 or rootindex>0:
            for priorbox in links[rootindex+i-1]:
                for delete_ind in deletes:
                    if delete_ind in priorbox:
                        priorbox.remove(delete_ind)
                        

def dsnms(preds):
    dets = [[]]
    for detection in preds:
        fid = []
        for idx, row in detection.pandas().xyxy[0].iterrows():
            fid.append([row.xmin, row.ymin, row.xmax, row.ymax, row.confidence])
        dets[0].append(fid)
    dets = np.array(dets)
    links=createLinks(dets)
    maxPath(dets,links)
    return dets

In [ ]:
preds = []

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    r = model(img, size=3600, augment=True)
    preds.append(r)
    preds = preds[-2:]
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    elif idx == 0:
        detecs = []
        scores = []
        for idx, row in r.pandas().xyxy[0].iterrows():
            detecs.append([row.xmin, row.ymin, row.xmax, row.ymax])
            scores.append(row.confidence)
        detecs, scores = nms(detecs, scores, IOU_THRESH)
        for i, det in enumerate(detecs):
            if scores[i] > 0.25:
                anno += '{} {} {} {} {} '.format(scores[i], int(det[0]), int(det[1]), int(det[2] - det[0]), int(det[3] - det[1]))
    
    else:
        pre_nms_dets = dsnms(preds)
        correct_frame = pre_nms_dets[0][1]
        detecs, scores = nms(correct_frame[:, :4], correct_frame[:, 4], IOU_THRESH)
        for i, det in enumerate(detecs):
            if scores[i] > 0.25:
                anno += '{} {} {} {} {} '.format(scores[i], int(det[0]), int(det[1]), int(det[2] - det[0]), int(det[3] - det[1]))
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)